In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import os
import re


In [2]:
path='//192.168.14.50/Public/ВЫГРУЗКИ/'
files=os.listdir(path)


In [3]:

def shop_name(shop):

    m1 = re.compile('([0-9]{2}-[0-9]{,2}-)([0-9]{,3})')
    m = m1.findall(shop)
    for i in range(len(m)):
        m[i] = m[i][0] + ('000' + m[i][1])[-3:]
    return m[0]

In [4]:
# arr=[]
# with open(path + 'ПРОДАЖИ ПО ДНЯМ (таблица для анализа) от 04-09-24.txt','r',encoding='utf8') as f:
#   arr.append(f.readlines())
ds=pd.read_csv(path + 'ПРОДАЖИ ПО ДНЯМ (таблица для анализа) от 04-09-24.txt', sep='\t')
ds['Выручка,']=ds['Выручка,'].str.replace(',','.')
ds['Выручка,']=ds['Выручка,'].str.replace(chr(160),'')
ds['Количество']=ds['Количество'].str.replace(',','.')
ds['Выручка,']=pd.to_numeric(ds['Выручка,'])
ds['День.Начало дня']=pd.to_datetime(ds['День.Начало дня'],dayfirst=True,errors='coerce')
ds=ds[ds['День.Начало дня'].isna()==False]
ds=ds.assign(Номер_тт= ds['Склад / магазин'].apply(shop_name))
ds[['Код','Номенклатура']]=ds['Код, Номенклатура'].str.split(',',n=1,expand=True)
ds['Выручка,'].fillna(0)
ds['Количество']=pd.to_numeric(ds['Количество'])
ds= ds.dropna(subset=['Количество'])
ds=ds.drop(['Код, Номенклатура'],axis=1)
ds['Месяц']= pd.DatetimeIndex(ds['День.Начало дня']).month
ds['Год']= pd.DatetimeIndex(ds['День.Начало дня']).year

In [5]:
ds.info()

<class 'pandas.core.frame.DataFrame'>
Index: 89839 entries, 0 to 89841
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   День.Начало дня  89839 non-null  datetime64[ns]
 1   Склад / магазин  89839 non-null  object        
 2   Продавец         89825 non-null  object        
 3   Характеристика   44180 non-null  object        
 4   Количество       89839 non-null  float64       
 5   Выручка,         89839 non-null  float64       
 6   Номер_тт         89839 non-null  object        
 7   Код              89839 non-null  object        
 8   Номенклатура     89839 non-null  object        
 9   Месяц            89839 non-null  int32         
 10  Год              89839 non-null  int32         
dtypes: datetime64[ns](1), float64(2), int32(2), object(6)
memory usage: 7.5+ MB


In [6]:
ds.describe(include='all')

,День.Начало дня,Склад / магазин,Продавец,Характеристика,Количество,"Выручка,",Номер_тт,Код,Номенклатура,Месяц,Год
count,89839,89839,89825,44180,89839.000000,89839.000000,89839,89839,89839,89839.0,89839.0
unique,NaN,194,392,68,NaN,NaN,194,970,970,NaN,NaN
top,NaN,"64-3-092 Саратов,пл.Орджоникидзе, 1(Оранжевый)...",Баркова Ирина Анатольевна,70x50,NaN,NaN,64-3-092,НФ-00001439,Пакет Майка,NaN,NaN
freq,NaN,1421,803,5825,NaN,NaN,1421,2701,2701,NaN,NaN
mean,2024-08-16 02:27:47.062189312,NaN,NaN,NaN,1.446042,704.545209,NaN,NaN,NaN,8.0,2024.0
min,2024-08-01 00:00:00,NaN,NaN,NaN,-4.000000,-5299.000000,NaN,NaN,NaN,8.0,2024.0
25%,2024-08-09 00:00:00,NaN,NaN,NaN,1.000000,199.000000,NaN,NaN,NaN,8.0,2024.0
50%,2024-08-16 00:00:00,NaN,NaN,NaN,1.000000,458.000000,NaN,NaN,NaN,8.0,2024.0
75%,2024-08-23 00:00:00,NaN,NaN,NaN,1.000000,958.000000,NaN,NaN,NaN,8.0,2024.0
max,2024-08-31 00:00:00,NaN,NaN,NaN,40.000000,10990.000000,NaN,NaN,NaN,8.0,2024.0


In [7]:
ds_prod=ds[['Номенклатура','Код','Характеристика']].drop_duplicates().reset_index(drop=True)
ds_prod

,Номенклатура,Код,Характеристика
0,"Одеяло ""Лебединое озеро"", Демисезонное, в сумке",НФ-00000641,"1,5 сп."
1,"Одеяло ""Лебединое озеро"", Демисезонное, в сумке",НФ-00000641,Евро
2,"Полотенце махровое , ручное, однотонное в асс...",НФ-00000980,NaN
3,"Подушка ""Шелковый рай"", в сумке",НФ-00000583,70x50
4,"Одеяло ""Лебединое озеро"", Зимнее, в сумке",НФ-00000643,2 сп.
...,...,...,...
1482,"Пододеяльник, Сатин, геометрия",НФ-00004843,евро 240*215
1483,"Пододеяльник, Поплин, цветы",НФ-00004170,Евро 220*200
1484,"Наволочка, Сатин",НФ-00000642,50*70
1485,"Простыня, Полисатин, 2301-16",НФ-00004969,"1,5 сп 215х147"


In [8]:
ds_shops=ds[['Склад / магазин','Номер_тт']].drop_duplicates().reset_index(drop=True)

In [9]:
ds_shops

,Склад / магазин,Номер_тт
0,"18-3-000 Ижевск, ул. Ворошилова, 79 а (18-ИЖЕВСК)",18-3-000
1,"18-3-358 Ижевск, ул. Коммунаров, 204 (18-ИЖЕВСК)",18-3-358
2,"43-3-188 Киров, ул. Лепсе, 51 А (43-КИРОВ)",43-3-188
3,"43-3-189 Киров, ул.Московская,д.120,к.1 (43-КИ...",43-3-189
4,"43-3-191 Киров, ул.Горького,д.7А (43-КИРОВ)",43-3-191
...,...,...
189,"64-3-102 Саратов,ул.Тархова,29А/1(ТК Солнечный...",64-3-102
190,"73-3-097 Ульяновск,ул.Пионерская, 13/18 (73-УЛ...",73-3-097
191,"52-3-368 Н.Новгород, пл. Комсомольская, 6 (52-...",52-3-368
192,"63-3-026 Самара, ул.Ленинградская, 70 (Юность)...",63-3-026


In [12]:

ds=ds.reset_index(drop=True)

In [13]:
ds

,День.Начало дня,Склад / магазин,Продавец,Характеристика,Количество,"Выручка,",Номер_тт,Код,Номенклатура,Месяц,Год
0,2024-08-01,"18-3-000 Ижевск, ул. Ворошилова, 79 а (18-ИЖЕВСК)",Карачева Марина Михайловна,"1,5 сп.",1.0,1199.0,18-3-000,НФ-00000641,"Одеяло ""Лебединое озеро"", Демисезонное, в сумке",8,2024
1,2024-08-01,"18-3-000 Ижевск, ул. Ворошилова, 79 а (18-ИЖЕВСК)",Карачева Марина Михайловна,Евро,1.0,1699.0,18-3-000,НФ-00000641,"Одеяло ""Лебединое озеро"", Демисезонное, в сумке",8,2024
2,2024-08-01,"18-3-000 Ижевск, ул. Ворошилова, 79 а (18-ИЖЕВСК)",Карачева Марина Михайловна,NaN,1.0,179.0,18-3-000,НФ-00000980,"Полотенце махровое , ручное, однотонное в асс...",8,2024
3,2024-08-01,"18-3-358 Ижевск, ул. Коммунаров, 204 (18-ИЖЕВСК)",Яруллина Резида Равильевна,70x50,2.0,1398.0,18-3-358,НФ-00000583,"Подушка ""Шелковый рай"", в сумке",8,2024
4,2024-08-01,"18-3-358 Ижевск, ул. Коммунаров, 204 (18-ИЖЕВСК)",Яруллина Резида Равильевна,2 сп.,1.0,1799.0,18-3-358,НФ-00000643,"Одеяло ""Лебединое озеро"", Зимнее, в сумке",8,2024
...,...,...,...,...,...,...,...,...,...,...,...
89834,2024-08-31,"73-3-352 Ульяновск, пр. Хо Ши Мина, 19 (73-УЛЬ...",Семенюк Татьяна Лукашевна,NaN,2.0,20.0,73-3-352,НФ-00001439,Пакет Майка,8,2024
89835,2024-08-31,"73-3-352 Ульяновск, пр. Хо Ши Мина, 19 (73-УЛЬ...",Семенюк Татьяна Лукашевна,NaN,4.0,100.0,73-3-352,НФ-00003776,Салфетка Цветы,8,2024
89836,2024-08-31,"73-3-352 Ульяновск, пр. Хо Ши Мина, 19 (73-УЛЬ...",Семенюк Татьяна Лукашевна,NaN,1.0,249.0,73-3-352,НФ-00004825,"Футболка с принтом,муж.",8,2024
89837,2024-08-31,"73-3-352 Ульяновск, пр. Хо Ши Мина, 19 (73-УЛЬ...",Семенюк Татьяна Лукашевна,2-х сп.,1.0,1499.0,73-3-352,НФ-00004834,"КПБ, ""Vernisage"", satin soft , 2309-3",8,2024
